# Pre-study: Manual Generation of Conversion Code


In this study, you will be presented with 30 tasks within Google Colab, divided into five sets to cover a range of complexities. Each set will contain 6 tasks.

Before you begin, please ensure you enable GPU hardware acceleration to optimize your performance. This can be done by navigating to "Edit" > "Notebook settings" and selecting "GPU" as the hardware accelerator.

Upon completing the tasks, you are encouraged to run the provided unit tests. These tests will help you assess the accuracy of your solutions and provide detailed feedback on any tasks that have failed.

To conclude your participation, please download the Jupyter notebook by going to the "File" menu and selecting "Download". This will allow you to save a copy of your work and any notes you may have made during the study.

In [ ]:
import sys
import os


if 'google.colab' in sys.modules:
  repo_url = 'https://github.com/c3di/ImgDataConvertCodeGen_Evaluation.git'
  !git clone {repo_url}
  repo_dir = repo_url.split('/')[-1].split('.git')[0]
  os.chdir(repo_dir)

In [25]:
import numpy as np
import torch
import tensorflow as tf
from PIL import Image
from torchvision.transforms import functional as F
from src.test_util import test_conversion_functions

## One Step Covertion

In [3]:
# When use scikit-image and opencv-image
def numpy_rgb_to_opencv_bgr(image):
    return image[::-1, :, :]

In [4]:
# When use scikit-image
def numpy_channel_first_to_last(image):
    return np.transpose(image, (1, 2, 0))

In [5]:
def tf_batched_to_unbatched(image):
    return tf.squeeze(image, 0)

In [6]:
def torch_uint8_to_float0to1(image):
    return image / 255.0

In [7]:
def torch_gpu_to_cpu(image):
    return image.cpu()

In [8]:
def pil_to_tf(image):
    return tf.convert_to_tensor(image)

In [9]:
# run tests
tests_config = [
    ({"data_representation": "numpy.ndarray", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     {"data_representation": "numpy.ndarray", "color_channel": 'bgr', "channel_order": 'channel first',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'}, 
     numpy_rgb_to_opencv_bgr),

    ({"data_representation": "numpy.ndarray", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     {"data_representation": "numpy.ndarray", "color_channel": 'rgb', "channel_order": 'channel last',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     numpy_channel_first_to_last),

    ({"data_representation": "tf.tensor", "color_channel": 'rgb', "channel_order": 'channel last',
      "minibatch_input": True, "image_data_type": 'uint8', "device": 'cpu'},
     {"data_representation": "tf.tensor", "color_channel": 'rgb', "channel_order": 'channel last',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     tf_batched_to_unbatched),

    ({"data_representation": "torch.tensor", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     {"data_representation": "torch.tensor", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": False, "image_data_type": 'float32(0to1)', "device": 'cpu'},
     torch_uint8_to_float0to1),

    ({"data_representation": "torch.tensor", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'gpu'},
     {"data_representation": "torch.tensor", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     torch_gpu_to_cpu),

    ({"data_representation": "PIL.Image", "color_channel": 'rgb', "channel_order": 'channel last',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     {"data_representation": "tf.tensor", "color_channel": 'rgb', "channel_order": 'channel last',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     pil_to_tf)]

test_conversion_functions(tests_config)


Passed/Total: 6/6


## Two Steps Convertion

In [10]:
def numpy_float0to1_to_uint8(image):
    return (image * 255).astype(np.uint8)

In [11]:
def tf_rgb_to_gray_nonchannel(image):
    return tf.squeeze(tf.image.rgb_to_grayscale(image), -1)

In [12]:
def tf_gray_nonechannel_float32_to_channellast_float64(image):
    return tf.expand_dims(tf.image.convert_image_dtype(image, tf.float64), -1)

In [13]:
def torch_channelfirst_unbatched_to_channlelast_batched(image):
    return image.permute(1, 2, 0).unsqueeze(0)

In [14]:
def torch_rgb_from_batched_cpu_to_unbatched_gpu(image):
    return image.squeeze(0).cuda()

In [15]:
def numpy_gray_with_channal_last_to_pil_rgb(image):
    return Image.fromarray(np.repeat(image, 3, axis=2))

In [16]:
# run tests
tests_config = [
    ({"data_representation": "numpy.ndarray", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": False, "image_data_type": 'float32(0to1)', "device": 'cpu'},
     {"data_representation": "numpy.ndarray", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     numpy_float0to1_to_uint8),

    ({"data_representation": "tf.tensor", "color_channel": 'rgb', "channel_order": 'channel last',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     {"data_representation": "tf.tensor", "color_channel": 'gray', "channel_order": 'none',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     tf_rgb_to_gray_nonchannel),

    ({"data_representation": "tf.tensor", "color_channel": 'gray', "channel_order": 'none',
      "minibatch_input": False, "image_data_type": 'float32(0to1)', "device": 'cpu'},
     {"data_representation": "tf.tensor", "color_channel": 'gray', "channel_order": 'channel last',
      "minibatch_input": False, "image_data_type": 'float64(0to1)', "device": 'cpu'},
     tf_gray_nonechannel_float32_to_channellast_float64),

    ({"data_representation": "tf.tensor", "color_channel": 'gray', "channel_order": 'none',
      "minibatch_input": False, "image_data_type": 'float32(0to1)', "device": 'cpu'},
     {"data_representation": "tf.tensor", "color_channel": 'gray', "channel_order": 'channel last',
      "minibatch_input": False, "image_data_type": 'float64(0to1)', "device": 'cpu'},
     tf_gray_nonechannel_float32_to_channellast_float64),

    ({"data_representation": "torch.tensor", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": True, "image_data_type": 'float32(0to1)', "device": 'cpu'},
     {"data_representation": "torch.tensor", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": False, "image_data_type": 'float32(0to1)', "device": 'gpu'},
     torch_rgb_from_batched_cpu_to_unbatched_gpu),

    ({"data_representation": "numpy.ndarray", "color_channel": 'gray', "channel_order": 'channel last',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     {"data_representation": "PIL.Image", "color_channel": 'rgb', "channel_order": 'channel last',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     numpy_gray_with_channal_last_to_pil_rgb),
]

test_conversion_functions(tests_config)


Passed/Total: 6/6


## Three Steps Conversion

In [17]:
# When use opencv-python and scikit-image
def numpy_bgr_uint8_to_rgb_float(image):
    return (image[::-1,:,:] * 255).astype(np.uint8)

In [18]:
def numpy_with_channelfirst_float_from_rgb_unbatched_to_gray_batched(image):
    weight = np.array([0.299, 0.587, 0.114]).reshape(3, 1, 1)
    converted = np.sum(image * weight, axis=0, keepdims=True)
    return converted[np.newaxis, ...]

In [19]:
def tf_rgb_channelfirst_batched_to_gray_channellast_unbatched(image):
    converted = tf.squeeze(image, 0)
    converted = tf.transpose(converted, [1, 2, 0])
    return tf.image.rgb_to_grayscale(converted)

In [20]:
def tf_gray_uint8_nonechannel_to_rgb_float320to1_channellast(image):
    converted = tf.image.convert_image_dtype(image, tf.float32)
    converted = tf.expand_dims(converted, -1)
    return tf.image.grayscale_to_rgb(converted)

In [21]:
def torch_channelfirst_float32_to_channellast_uint8(image):
    converted = (image * 255).to(torch.uint8)
    return converted.permute(1, 2, 0)

In [22]:
def torch_rgb_gpu_to_numpy_gray_cpu(image):
    converted = F.rgb_to_grayscale(image.cpu())
    return converted.numpy(force=True)

In [23]:
# run tests
tests_config = [
    ({"data_representation": "numpy.ndarray", "color_channel": 'bgr', "channel_order": 'channel first',
      "minibatch_input": False, "image_data_type": 'float32(0to1)', "device": 'cpu'},
     {"data_representation": "numpy.ndarray", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     numpy_bgr_uint8_to_rgb_float),

    ({"data_representation": "numpy.ndarray", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": False, "image_data_type": 'float32(0to1)', "device": 'cpu'},
     {"data_representation": "numpy.ndarray", "color_channel": 'gray', "channel_order": 'channel first',
      "minibatch_input": True, "image_data_type": 'float32(0to1)', "device": 'cpu'},
     numpy_with_channelfirst_float_from_rgb_unbatched_to_gray_batched),

    ({"data_representation": "tf.tensor", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": True, "image_data_type": 'uint8', "device": 'cpu'},
     {"data_representation": "tf.tensor", "color_channel": 'gray', "channel_order": 'channel last',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     tf_rgb_channelfirst_batched_to_gray_channellast_unbatched),

    ({"data_representation": "tf.tensor", "color_channel": 'gray', "channel_order": 'none',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     {"data_representation": "tf.tensor", "color_channel": 'rgb', "channel_order": 'channel last',
      "minibatch_input": False, "image_data_type": 'float32(0to1)', "device": 'cpu'}, 
     tf_gray_uint8_nonechannel_to_rgb_float320to1_channellast),

    ({"data_representation": "torch.tensor", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": False, "image_data_type": 'float32(0to1)', "device": 'cpu'},
     {"data_representation": "torch.tensor", "color_channel": 'rgb', "channel_order": 'channel last',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     torch_channelfirst_float32_to_channellast_uint8),

    ({"data_representation": "torch.tensor", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'gpu'},
     {"data_representation": "numpy.ndarray", "color_channel": 'gray', "channel_order": 'channel first',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     torch_rgb_gpu_to_numpy_gray_cpu),
]

test_conversion_functions(tests_config)


Passed/Total: 6/6


## Four Steps Conversion

In [44]:
def numpy_gray_nonechannel_unbatched_uint8_to_rgb_channelfirst_batched_float0to1(image):
    converted = np.expand_dims(image, axis=0)
    converted = np.repeat(converted, 3, axis=0)
    converted = converted[np.newaxis, ...]
    return converted / 255.0    

In [45]:
def numpy_rgb_channelfirst_batched_float0to1_to_rgb_channellast_unbatched_uint8(image):
    converted = image.squeeze(0)
    converted = converted * 255
    converted = converted.astype(np.uint8)
    return converted.transpose(1, 2, 0)

In [87]:
def tf_rgb_uint8_unbatched_channellast_to_gray_float320to1_batched_nonechannel(image):
    converted = tf.image.rgb_to_grayscale(image)
    converted = tf.image.convert_image_dtype(converted, tf.float32)
    converted = tf.squeeze(converted, -1)
    return tf.expand_dims(converted, 0)

In [74]:
def tf_gray_float32_channellast_batched_to_rgb_chanelfirst_unit8_unbatched(image):
    converted = tf.squeeze(image, 0)
    converted = tf.image.grayscale_to_rgb(converted)
    converted = tf.transpose(converted, [2, 0, 1])
    return tf.image.convert_image_dtype(converted, tf.uint8)

In [81]:
def torch_rgb_uint8_batched_channelfirst_gpu_to_float_unbatched_channellast_cpu(image):
    converted = image.cpu()
    converted = converted / 255.0
    converted = converted.squeeze(0)
    return converted.permute(1, 2, 0)

In [179]:
# Will fixed by image data generator
def torch_rgb_channelfirst_batched_float_to_tf_gray_batched_channellast_float(image):
    converted = image.permute(0, 2, 3, 1)
    converted = converted.numpy()
    converted = tf.convert_to_tensor(converted)
    return tf.image.rgb_to_grayscale(converted)

In [161]:
# run tests
tests_config = [
    ({"data_representation": "numpy.ndarray", "color_channel": 'gray', "channel_order": 'none',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     {"data_representation": "numpy.ndarray", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": True, "image_data_type": 'float32(0to1)', "device": 'cpu'},
     numpy_gray_nonechannel_unbatched_uint8_to_rgb_channelfirst_batched_float0to1),

    ({"data_representation": "numpy.ndarray", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": True, "image_data_type": 'float32(0to1)', "device": 'cpu'},
     {"data_representation": "numpy.ndarray", "color_channel": 'rgb', "channel_order": 'channel last',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     numpy_rgb_channelfirst_batched_float0to1_to_rgb_channellast_unbatched_uint8),

    ({"data_representation": "tf.tensor", "color_channel": 'rgb', "channel_order": 'channel last',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     {"data_representation": "tf.tensor", "color_channel": 'gray', "channel_order": 'none',
      "minibatch_input": True, "image_data_type": 'float32(0to1)', "device": 'cpu'},
     tf_rgb_uint8_unbatched_channellast_to_gray_float320to1_batched_nonechannel),

    ({"data_representation": "tf.tensor", "color_channel": 'gray', "channel_order": 'channel last',
      "minibatch_input": True, "image_data_type": 'float32(0to1)', "device": 'cpu'},
     {"data_representation": "tf.tensor", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'}, 
     tf_gray_float32_channellast_batched_to_rgb_chanelfirst_unit8_unbatched),

    ({"data_representation": "torch.tensor", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": True, "image_data_type": 'uint8', "device": 'gpu'},
     {"data_representation": "torch.tensor", "color_channel": 'rgb', "channel_order": 'channel last',
      "minibatch_input": False, "image_data_type": 'float32(0to1)', "device": 'cpu'},
     torch_rgb_uint8_batched_channelfirst_gpu_to_float_unbatched_channellast_cpu),
    
    # ({"data_representation": "torch.tensor", "color_channel": 'rgb', "channel_order": 'channel first',
    #   "minibatch_input": True, "image_data_type": 'float32(0to1)', "device": 'cpu'},
    #  {"data_representation": "tf.tensor", "color_channel": 'gray', "channel_order": 'channel last',
    #   "minibatch_input": True, "image_data_type": 'float32(0to1)', "device": 'cpu'},
    #  torch_rgb_channelfirst_batched_float_to_tf_gray_batched_channellast_float),
]

test_conversion_functions(tests_config)


Passed/Total: 5/5


## More Steps Conversion

In [104]:
def numpy_bgr_channelfirst_unbatched_uint8_to_tf_rgb_channellast_batched_float0to1(image):
    converted = image[::-1, :, :]
    converted = converted.transpose(1, 2, 0)
    converted = np.expand_dims(converted, axis=0)
    converted = converted / 255.0 # np.float64
    return tf.convert_to_tensor(converted, dtype=tf.float32)

In [107]:
def numpy_rgb_channelfirst_batched_float0to1_to_PIL_rgb_channellast_unbatched_uint8(image):
    converted = image.squeeze(0)
    converted = converted * 255
    converted = converted.astype(np.uint8)
    converted = converted.transpose(1, 2, 0)
    return Image.fromarray(converted)

In [175]:
# Will fixed by image data generator
def PIL_gray_nonechannel_unit8_to_tf_rgb_float32_channellast_batched(image):
    converted = tf.convert_to_tensor(image)
    converted = tf.image.convert_image_dtype(converted, tf.float32)
    converted = tf.expand_dims(converted, -1)
    converted = tf.expand_dims(converted, 0)
    return tf.image.grayscale_to_rgb(converted)

In [164]:
# Will test tensorflow GPU
def tf_gray_channellast_batced_gpu_to_torch_rgb_channelfirst_batched_cpu(image):
    with tf.device('/cpu:0'):
        converted = tf.identity(image)
    converted = converted.numpy()
    converted = np.repeat(converted, 3, axis=-1)
    converted = np.transpose(tf_image_rgb, (0, 3, 1, 2))
    return torch.from_numpy()

In [165]:
# Will fix order of converted = converted / 255.0 converted = F.rgb_to_grayscale(converted)
def torch_rgb_uint8_batched_gpu_to_numpy_gray_float_batched_nonechannel_cpu(image):
    converted = image.cpu()
    converted = F.rgb_to_grayscale(converted)
    converted = converted / 255.0
    converted = converted.squeeze(1)
    return converted.numpy(force=True)

In [172]:
# Will fixed by image data generator
def torch_channelfirst_gray_float32_batched_gpu_to_tf_gray_channellast_uint8_nonnchannel_gpu(image):
    converted = image.cpu()
    converted = (converted * 255).to(torch.uint8)
    converted = converted.permute(0, 2, 3, 1)
    converted = converted.numpy()
    converted = tf.convert_to_tensor(numpy_image)
    converted = tf.squeeze(converted, -1)
    with tf.device('/gpu:0'):
        return tf.identity(var)

In [176]:
# run tests
tests_config = [
    ({"data_representation": "numpy.ndarray", "color_channel": 'bgr', "channel_order": 'channel first',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     {"data_representation": "tf.tensor", "color_channel": 'rgb', "channel_order": 'channel last',
      "minibatch_input": True, "image_data_type": 'float32(0to1)', "device": 'cpu'},
     numpy_bgr_channelfirst_unbatched_uint8_to_tf_rgb_channellast_batched_float0to1),

    ({"data_representation": "numpy.ndarray", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": True, "image_data_type": 'float32(0to1)', "device": 'cpu'},
     {"data_representation": "PIL.Image", "color_channel": 'rgb', "channel_order": 'channel last',
      "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
     numpy_rgb_channelfirst_batched_float0to1_to_PIL_rgb_channellast_unbatched_uint8),

    # ({"data_representation": "PIL.Image", "color_channel": 'gray', "channel_order": 'none',
    #   "minibatch_input": False, "image_data_type": 'uint8', "device": 'cpu'},
    #  {"data_representation": "tf.tensor", "color_channel": 'rgb', "channel_order": 'channel last',
    #   "minibatch_input": True, "image_data_type": 'float32(0to1)', "device": 'cpu'},
    #  PIL_gray_nonechannel_unit8_to_tf_rgb_float32_channellast_batched),

    # ({"data_representation": "tf.tensor", "color_channel": 'gray', "channel_order": 'channel last',
    #   "minibatch_input": True, "image_data_type": 'float32(0to1)', "device": 'gpu'},
    #  {"data_representation": "tf.tensor", "color_channel": 'rgb', "channel_order": 'channel first',
    #   "minibatch_input": True, "image_data_type": 'float32(0to1)', "device": 'cpu'},
    #  tf_gray_channellast_batced_gpu_to_torch_rgb_channelfirst_batched_cpu),
    
    ({"data_representation": "torch.tensor", "color_channel": 'rgb', "channel_order": 'channel first',
      "minibatch_input": True, "image_data_type": 'uint8', "device": 'gpu'},
     {"data_representation": "numpy.ndarray", "color_channel": 'gray', "channel_order": 'none',
      "minibatch_input": True, "image_data_type": 'float32(0to1)', "device": 'cpu'},
     torch_rgb_uint8_batched_gpu_to_numpy_gray_float_batched_nonechannel_cpu),

    # ({"data_representation": "torch.tensor", "color_channel": 'gray', "channel_order": 'channel first',
    #   "minibatch_input": True, "image_data_type": 'float32(0to1)', "device": 'gpu'},
    #  {"data_representation": "tf.tensor", "color_channel": 'gray', "channel_order": 'channel last',
    #   "minibatch_input": True, "image_data_type": 'uint8', "device": 'cpu'},
    #  torch_channelfirst_gray_float32_batched_gpu_to_tf_gray_channellast_uint8_nonnchannel_gpu),
]

test_conversion_functions(tests_config)


Passed/Total: 3/3
